In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [70]:
def process_data(df, drop_customers = True):
    df['Date'] = pd.to_datetime(df['Date']) # convert to datetime

    df['Date2'] = np.datetime64('2013-01-01')
    df['Days'] = df['Date'] - df['Date2']
    df['Days'] = (df['Days'] / np.timedelta64(1, 'D')).astype(int)

    #take 'StateHoliday and divide it into A, B, C, and True values (0 or 1)'
    df['StateHolidayA'] = 0 
    df['StateHolidayB'] = 0
    df['StateHolidayC'] = 0
    df['StateHolidayTrue'] = 0
    for i,val in enumerate(df.StateHoliday):
        if val == 'a':
            df.set_value(i, 'StateHolidayA', 1)
            df.set_value(i, 'StateHolidayB', 0)
            df.set_value(i, 'StateHolidayC', 0)
            df.set_value(i, 'StateHolidayTrue', 1)
        elif val == 'b':
            df.set_value(i, 'StateHolidayA', 0)
            df.set_value(i, 'StateHolidayB', 1)
            df.set_value(i, 'StateHolidayC', 0)
            df.set_value(i, 'StateHolidayTrue', 1)
        elif val == 'c':
            df.set_value(i, 'StateHolidayA', 0)
            df.set_value(i, 'StateHolidayB', 0)
            df.set_value(i, 'StateHolidayC', 1)
            df.set_value(i, 'StateHolidayTrue', 1)
        else:
            df.set_value(i, 'StateHolidayA', 0)
            df.set_value(i, 'StateHolidayB', 0)
            df.set_value(i, 'StateHolidayC', 0)
            df.set_value(i, 'StateHolidayTrue', 0)
    
    df.drop(['StateHoliday','Date','Date2'], axis = 1, inplace=True)
    if drop_customers == True:
        try:
            df.drop('Customers', axis = 1, inplace=True)
        except:
            pass
    try:
        df.drop('Id', axis = 1, inplace=True)
    except:
        pass
    
    return df

In [37]:
df = pd.read_csv('data/train.csv')
y_customers = df['Customers']
df = process_data(df)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 11 columns):
Store               1017209 non-null int64
DayOfWeek           1017209 non-null int64
Sales               1017209 non-null int64
Open                1017209 non-null int64
Promo               1017209 non-null int64
SchoolHoliday       1017209 non-null int64
Days                1017209 non-null int64
StateHolidayA       1017209 non-null int64
StateHolidayB       1017209 non-null int64
StateHolidayC       1017209 non-null int64
StateHolidayTrue    1017209 non-null int64
dtypes: int64(11)
memory usage: 93.1 MB


In [5]:
df.head()

,Store,DayOfWeek,Sales,Open,Promo,SchoolHoliday,Days,StateHolidayA,StateHolidayB,StateHolidayC,StateHolidayTrue
0,1,5,5263,1,1,1,941,0,0,0,0
1,2,5,6064,1,1,1,941,0,0,0,0
2,3,5,8314,1,1,1,941,0,0,0,0
3,4,5,13995,1,1,1,941,0,0,0,0
4,5,5,4822,1,1,1,941,0,0,0,0


In [6]:
y = df['Sales']
X = df.drop('Sales', axis = 1)
print X.shape, y.shape

(1017209, 10) (1017209,)


In [7]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
# from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_score, accuracy_score, r2_score
# rf = RandomForestRegressor(oob_score=True)
# lr = LinearRegression()
etr = ExtraTreesRegressor(n_estimators = 100, oob_score=True, bootstrap = True, warm_start = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
etr.fit(X_train, y_train)
y_pred = etr.predict(X_test)
print 'oob score: ', etr.oob_score_

oob score:  0.611714243883


In [9]:
df2 = pd.read_csv('data/test.csv')
df2 = process_data(df2)
df2.fillna(0,inplace=True)
for i,val in enumerate(X.columns):
    print X.columns[i], df2.columns[i]
df2.head()

Store Store
DayOfWeek DayOfWeek
Open Open
Promo Promo
SchoolHoliday SchoolHoliday
Days Days
StateHolidayA StateHolidayA
StateHolidayB StateHolidayB
StateHolidayC StateHolidayC
StateHolidayTrue StateHolidayTrue


,Store,DayOfWeek,Open,Promo,SchoolHoliday,Days,StateHolidayA,StateHolidayB,StateHolidayC,StateHolidayTrue
0,1,4,1,1,0,989,0,0,0,0
1,3,4,1,1,0,989,0,0,0,0
2,7,4,1,1,0,989,0,0,0,0
3,8,4,1,1,0,989,0,0,0,0
4,9,4,1,1,0,989,0,0,0,0


In [10]:
sorted_mask = np.argsort(etr.feature_importances_)[::-1] #feature importances in descending order

for i in zip(X.columns[sorted_mask], etr.feature_importances_[sorted_mask]):
    print i

('Open', 0.46004617510543361)
('Store', 0.38788911861331832)
('Promo', 0.073472455582667268)
('Days', 0.051730537676889982)
('DayOfWeek', 0.023767415023816397)
('SchoolHoliday', 0.0024006670124643308)
('StateHolidayB', 0.00029114347151221409)
('StateHolidayTrue', 0.00024012406583735421)
('StateHolidayA', 0.00012432238339651303)
('StateHolidayC', 3.8041064664232774e-05)


In [11]:
sales_pred = etr.predict(df2)

In [12]:
print len(np.unique(sales_pred))
np.unique(sales_pred)

15247


array([     0.  ,   1738.21,   2407.69, ...,  19430.49,  19614.63,
        30845.94])

In [13]:
len(np.unique(df.Store))

1115

In [14]:
df3 = pd.DataFrame.from_dict({'Id':np.arange(1,len(sales_pred)+1)})
df3['Sales'] = sales_pred
df3.head()

,Id,Sales
0,1,7936.32
1,2,8416.20
2,3,8820.49
3,4,8597.44
4,5,8498.99


In [15]:
df3.to_csv('data/submission.csv', index=False)

##Add customers to model
###predict number of "virtual customers"

In [16]:
etr2 = ExtraTreesRegressor(n_estimators = 50, oob_score=True, bootstrap = True, warm_start = True)
#from sklearn.linear_model import LinearRegression
#lr = LinearRegression()
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y_customers, test_size=0.33, random_state=42)

In [17]:
etr2.fit(X_train2, y_train2)
y_pred2 = etr2.predict(X_test2)
print 'oob score: ', etr2.oob_score_

oob score:  0.783385638672


In [18]:
print np.mean(y_pred2)
print np.mean(y_test2)
print r2_score(y_test2,y_pred2)
print y_test2.values[55], y_pred2[55]


632.186366797
633.658125769
0.793606706474
689 701.38


###use virtual customers to help predict sales

In [66]:
etr2.predict(X.values[2])[0]

822.86000000000001

In [59]:
#X.drop('Customers',axis=1,inplace=True)
X.shape

(1017209, 10)

In [67]:
#add virtual customers
def add_customers(df, model):
#     df_temp = df.copy()
#     df_temp['Customers'] = 1
    X_temp = df.values
    myArray = [None]*len(X_temp)
    for i,val in enumerate(X_temp):
        myArray[i] = model.predict(X_temp[i])[0]
    return np.array(myArray)

In [ ]:
#print add_customers(X, etr2)

In [ ]:
df_sales = pd.read_csv('data/train.csv')
df_sales = process_data(df_sales, drop_customers=False)

In [84]:
y_sales = df_sales['Sales']
X_sales = df_sales.drop('Sales', axis = 1)
print X_sales.shape, y_sales.shape

(1017209, 11) (1017209,)


In [85]:
etr3 = ExtraTreesRegressor(n_estimators = 100, oob_score=True, bootstrap = True, warm_start = True)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_sales, y_sales, test_size=0.33, random_state=42)

In [86]:
etr3.fit(X_train3, y_train3)
y_pred3 = etr3.predict(X_test3)
print 'oob score: ', etr3.oob_score_

oob score:  0.906891246981


###Test data again

In [124]:
df2 = pd.read_csv('data/test.csv')
df2 = process_data(df2)
df2.fillna(1,inplace=True)

In [125]:
np.unique(df2.Open)

array([ 0.,  1.])

In [126]:
df2['Customers'] = add_customers(df2, etr2)
df2.head()

,Store,DayOfWeek,Open,Promo,SchoolHoliday,Days,StateHolidayA,StateHolidayB,StateHolidayC,StateHolidayTrue,Customers
0,1,4,1,1,0,989,0,0,0,0,567.60
1,3,4,1,1,0,989,0,0,0,0,770.02
2,7,4,1,1,0,989,0,0,0,0,909.00
3,8,4,1,1,0,989,0,0,0,0,850.56
4,9,4,1,1,0,989,0,0,0,0,657.14


In [127]:
myCustomers = df2.Customers.values
df2.drop('Customers',axis=1,inplace=True)

In [128]:
df2.insert(2, 'Customers' , myCustomers)

In [129]:
df2.columns

Index([u'Store', u'DayOfWeek', u'Customers', u'Open', u'Promo',
       u'SchoolHoliday', u'Days', u'StateHolidayA', u'StateHolidayB',
       u'StateHolidayC', u'StateHolidayTrue'],
      dtype='object')

In [130]:
sales_pred = etr3.predict(df2)

In [131]:
df3 = pd.DataFrame.from_dict({'Id':np.arange(1,len(sales_pred)+1)})
df3['Sales'] = sales_pred
df3.head(1000)

,Id,Sales
0,1,6139.21
1,2,7282.76
2,3,10314.31
3,4,8719.47
4,5,7173.17
5,6,7455.03
6,7,10204.83
7,8,7535.66
8,9,5872.33
9,10,7191.51


In [132]:
df3.to_csv('data/submission.csv', index=False)

##Add store data to model

In [ ]:
df_store = pd.read_csv('data/store.csv')

In [ ]:
df_store.head()

In [ ]:
df_store_lite = pd.DataFrame(df_store['Store'])
df_store_lite['StoreType'] = df_store['StoreType']
df_store_lite['Assortment'] = df_store['Assortment']
df_store_lite['CompetitionDistance'] = df_store['CompetitionDistance']
df_store_lite.head()

In [ ]:
pd.merge(df, df_store_lite, on='Store', how='left')